<span style="color:red; font-family:Helvetica Neue, Helvetica, Arial, sans-serif; font-size:2em;">An Exception was encountered at '<a href="#papermill-error-cell">In [3]</a>'.</span>

# Kubernetes Cluster Versions Report

This report loops through all the clusters we maintain, and reports their master and node versions

In [1]:
from deployer.helm_upgrade_decision import get_all_cluster_yaml_files
from deployer.cluster import Cluster
import subprocess
import json
from tabulate import tabulate

from ruamel.yaml import YAML
yaml = YAML(typ='safe')

In [2]:
cluster_files = get_all_cluster_yaml_files()

clusters = []
for cf in cluster_files:
    with open(cf) as f:
        cluster = Cluster(yaml.load(f), cf.parent)
    clusters.append(cluster)

<span id="papermill-error-cell" style="color:red; font-family:Helvetica Neue, Helvetica, Arial, sans-serif; font-size:2em;">Execution using papermill encountered an exception here and stopped:</span>

In [3]:
versions = {}
for c in clusters:
    with c.auth():
        cluster_name = c.spec['name']
        # Populate cloud provider type
        versions[cluster_name] = {'provider': c.spec['provider']}
        
        # Get master version
        version_output = json.loads(subprocess.check_output(['kubectl', 'version', '--output', 'json']))
        server_version = version_output["serverVersion"]["gitVersion"]
        versions[cluster_name]['master'] = server_version.split('-')[0]
        
        # Get node versions
        node_output = json.loads(subprocess.check_output(["kubectl", "get", "node", "--output", "json"]))
        node_versions = set([n["status"]["nodeInfo"]["kubeletVersion"].split('-')[0] for n in node_output["items"]])
        versions[cluster_name]['nodes'] = node_versions
        
        

Unable to connect to the server: getting credentials: exec: executable gke-gcloud-auth-plugin not found

It looks like you are trying to use a client-go credential plugin that is not installed.

To learn more about this feature, consult the documentation available at:
      https://kubernetes.io/docs/reference/access-authn-authz/authentication/#client-go-credential-plugins

Install gke-gcloud-auth-plugin for use with kubectl by following https://cloud.google.com/blog/products/containers-kubernetes/kubectl-auth-changes-in-gke


CalledProcessError: Command '['kubectl', 'version', '--output', 'json']' returned non-zero exit status 1.

In [ ]:
headers = ('provider', 'cluster', 'master version', 'node version(s)')
tabulate([
    (version['provider'], name, version['master'], ','.join(version['nodes']))
    for name, version in versions.items()
], headers, tablefmt="html")
